### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain

In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

In [3]:
!pip install langchain_groq

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F71CA08430>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F71C832E30>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_core

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [7]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a common and casual greeting among friends.)\n* **Coucou, comment vas-tu ?** (This is a more friendly and informal greeting, often used with close friends.) \n\nLet me know if you\'d like more options!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 21, 'total_tokens': 133, 'completion_time': 0.203636364, 'prompt_time': 0.002095173, 'queue_time': 0.258647242, 'total_time': 0.205731537}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--57f89ec8-81f2-4d4d-a83a-86a49a983fc5-0', usage_metadata={'input_tokens': 21, 'output_tokens': 112, 'total_tokens': 133})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a common and casual greeting among friends.)\n* **Coucou, comment vas-tu ?** (This is a more friendly and informal greeting, often used with close friends.) \n\nLet me know if you\'d like more options!\n'

In [9]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal)\n* **Bonjour, ça va ?** (Informal) \n\n\nLet me know if you\'d like to know more about French greetings!\n'

In [10]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [11]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [12]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [13]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [14]:
!pip install streamlit

  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.1 MB 2.4 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/10.1 MB 2.6 MB/s eta 0:00:04
   ------- -------------------------------- 1.8/10.1 MB 2.6 MB/s eta 0:00:04
   --------- ------------------------------ 2.4/10.1 MB 2.6 MB/s eta 0:00:03
   ------------- -------------------------- 3.4/10.1 MB 3.1 MB/s eta 0:00:03
   ---------------- ----------------------- 4.2/10.1 MB 3.1 MB/s eta 0:00:02
   ------------------ --------------------- 4.7/10.1 MB 3.0 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.1 MB 3.0 MB/s eta 0:00:02
   ----------------------- ---------------- 6.0/10.1 MB 3.1 MB/s eta 0:00:02
   ----------------------------- ---------- 7.3/10.1 MB 3.3 MB/s eta 0:00:01
   --------------------